In [1]:
import socket
import sys
import requests
import requests_oauthlib
import json
sock = None

In [2]:
with open("./accessKeys.json") as f:
    data = json.load(f)

In [3]:
ACCESS_TOKEN = data['ACCESS_TOKEN']
ACCESS_SECRET = data['ACCESS_SECRET']
CONSUMER_KEY = data['CONSUMER_KEY']
CONSUMER_SECRET = data['CONSUMER_SECRET']
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

In [62]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'es'), ('locations', '-9,35,3,42'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

In [5]:
def send_tweets_to_spark(http_resp, tcp_connection, limit=-1):
    for line in http_resp.iter_lines():
        limit -= 1
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            print("Tweet Text: " + tweet_text)
            print ("------------------------------------------")
            tcp_connection.send(bytes(tweet_text + '\n', encoding='utf8'))
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)
        print("Limit = " + str(limit))
        if limit == 0:
            http_resp.close()
            break

# Debug

In [61]:

def debug():
    TCP_IP = "localhost"
    TCP_PORT = 8008
    conn = None
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((TCP_IP, TCP_PORT))
    s.listen(1)
    print("Waiting for TCP connection...")
    conn, addr = s.accept()
    print("Connected... Starting getting tweets.")
    global sock
    sock = s
    resp = get_tweets()
    send_tweets_to_spark(resp, conn, limit=20)
    
debug()

Waiting for TCP connection...
Connected... Starting getting tweets.
https://stream.twitter.com/1.1/statuses/filter.json?language=es&locations=-9,35,3,42&track=#playstation5 <Response [200]>
Tweet Text: @lauraulet Nos pasa
------------------------------------------
Limit = 19
Tweet Text: @EricPLL Osea, para mí dios puede ser perfectamente lo que otros entienden por Naturaleza, o por Destino, o en la l… https://t.co/25oMTDS9Ot
------------------------------------------
Limit = 18
Tweet Text: @r_munozavia Maravilla 😄
------------------------------------------
Limit = 17
Tweet Text: Jardinero, Encofrador, Cerrajero, Mantenimiento...... Algún curro para mi???
------------------------------------------
Limit = 16
Tweet Text: @bballestin Hay mucha mier.....y encima explotacion por mafias que se dan de alta y subcontractan a otras personas.… https://t.co/qspokxWj9h
------------------------------------------
Limit = 15
Tweet Text: Por coherencia profesional y apoyo a nuestro club quedan prohibi

In [52]:
sock.close()